In [62]:
from collections import defaultdict
from multiprocessing import Pool, Queue
import multiprocessing
import datetime
import math
import time
import pandas as pd
import requests
import numpy as np
import json
import os
import torch

begin_date = '2020-01-01'
end_date = '2020-12-31'
AMODATA_DIR = 'amoData/'
AMOMAT_DIR = 'amoMat/'
OCCMAT_DIR = 'occMat/'
PERMAT_DIR = 'perMat/'
BETTI_DIR = 'betti/'
BETTI_0_DIR = 'betti/betti_0/'
BETTI_1_DIR = 'betti/betti_1/'

betti_0_total = pd.DataFrame([])
betti_1_total = pd.DataFrame([])

def getBetweenDay(begin_date, end_date):
    date_list = []
    date_arr = []
    date_unix_list = []
    begin_date = datetime.datetime.strptime(begin_date, "%Y-%m-%d")
    print(type(begin_date))
    print("begin_date:",begin_date)
    # end_date = datetime.datetime.strptime(time.strftime('%Y-%m-%d', time.localtime(time.time())), "%Y-%m-%d")
    end_date = datetime.datetime.strptime(end_date, "%Y-%m-%d")
    print("end_date:",end_date)
    while begin_date <= end_date:
        date_unix = math.trunc(begin_date.replace(tzinfo=datetime.timezone.utc).timestamp()*1000)
        date_unix_list.append(date_unix)
        date_str = begin_date.strftime("%Y-%m-%d")
        date_list.append(date_str)
        date_arr.append([date_str, date_unix])
        begin_date += datetime.timedelta(days=1)  
    return  np.asarray(date_arr)
def getYearFromDate(date):
    Year = date.split("-")[0]
    return Year

def getYearsFromDate(begin_date, end_date):
    begin_Year = begin_date.split("-")[0]
    end_Year = end_date.split("-")[0]
    YEARS = [str(i) for i in range(int(begin_Year), int(end_Year) + 1)]
    return YEARS 


amoData_mean_total = []
amoData_median_total = []
amoData_sum_total = []
amoData_std_total = []
amoData_tx_total = []
begin_date = '2020-01-01'
end_date = '2020-12-31'
for date in getBetweenDay(begin_date, end_date):
    amoData = pd.read_csv("/root/study/ChainMT/data/eth/amoData/2020/" + date[0] + ".csv", names=["0","1"], index_col=0, converters={"0":str,"1":float}).sort_values(by=['1']).reset_index(drop=True)
    amoData["1"] = amoData["1"] / 2
    #print("amoData:",amoData)
    amoData_totaltx_in = amoData["0"].str[0:2].astype(int).sum() - len(amoData)
    #print("amoData_totaltx:",amoData_totaltx_in)
    amoData_tx_total.append(amoData_totaltx_in)
    amoData = amoData["1"]
    amoData_mean = amoData.mean()
    amoData_mean_total.append(amoData_mean)
    amoData_median = amoData.median()
    amoData_median_total.append(amoData_median)
    amoData_sum = amoData.sum()
    amoData_sum_total.append(amoData_sum)
    amoData_std = amoData.std()
    amoData_std_total.append(amoData_std)

print(amoData_mean_total,amoData_mean_total)
df_eth = pd.read_csv("Ethereum Historical Data - Investing.com.csv", converters={"Date":str})
df_eth["Date"] =  pd.to_datetime(df_eth["Date"], format='%b %d, %Y')
df_eth["Ethereum"] = df_eth["Open"].str.replace(",","").astype(float)
#df_eth = df_eth.set_index("Date")
df_eth = df_eth[df_eth["Date"].astype(str).str.contains("2020")].loc[:,["Date","Open"]].reset_index(drop=True)
df_eth.columns = ["date", "price"]
amoData_total = pd.DataFrame([amoData_mean_total,amoData_median_total,amoData_sum_total,amoData_std_total,amoData_tx_total]).T
amoData_total.columns = ["meanValue", "medianValue", "sumValue", "stdValue","totaltx"]

amoData_total_new = pd.concat([amoData_total, df_eth],axis=1)
amoData_total_new.set_index('date',inplace=True)
amoData_total_new.to_csv("trade/ethereum_2020_all_features.csv")

<class 'datetime.datetime'>
begin_date: 2020-01-01 00:00:00
end_date: 2020-12-31 00:00:00
amoData:            0             1
0       0108  0.000000e+00
1       0201  0.000000e+00
2       0201  0.000000e+00
3       0201  0.000000e+00
4       0801  0.000000e+00
...      ...           ...
198707  2003  7.075276e+21
198708  2003  7.178510e+21
198709  2020  1.143740e+22
198710  2003  1.203325e+22
198711  2020  1.725684e+22

[198712 rows x 2 columns]
amoData_totaltx: 325347
amoData:            0             1
0       0106  0.000000e+00
1       0201  0.000000e+00
2       0201  0.000000e+00
3       0301  0.000000e+00
4       0102  0.000000e+00
...      ...           ...
220445  0201  2.464819e+22
220446  0102  2.464819e+22
220447  0201  2.500000e+22
220448  2020  3.140479e+22
220449  0703  3.289515e+22

[220450 rows x 2 columns]
amoData_totaltx: 361247
amoData:            0             1
0       0102  0.000000e+00
1       0201  0.000000e+00
2       0103  0.000000e+00
3       0201  0.000000e+0

In [72]:
df_eth = pd.read_csv("Ethereum Historical Data - Investing.com.csv", converters={"Date":str})
df_eth["Date"] =  pd.to_datetime(df_eth["Date"], format='%b %d, %Y')
df_eth["Ethereum"] = df_eth["Open"].str.replace(",","").astype(float)
#df_eth = df_eth.set_index("Date")
df_eth = df_eth[df_eth["Date"].astype(str).str.contains("2020")].loc[:,["Date","Open"]].reset_index(drop=True)
df_eth.columns = ["date", "price"]
amoData_total = pd.DataFrame([amoData_mean_total,amoData_median_total,amoData_sum_total,amoData_std_total,amoData_tx_total]).T
amoData_total.columns = ["meanValue", "medianValue", "sumValue", "stdValue","totaltx"]

amoData_total_new = pd.concat([amoData_total, df_eth],axis=1)
amoData_total_new.to_csv("trade/ethereum_2020_all_features.csv")

bitcoin_data = pd.read_csv("/root/study/ChainMT/thesis figure/trade/pricedBitcoin2020.csv",index_col=0).reset_index(drop=True)
amoData_total_new = pd.concat([amoData_total_new, bitcoin_data.loc[:,["year", "day"]]],axis=1)
amoData_total_new

,meanValue,medianValue,sumValue,stdValue,totaltx,date,price,year,day
0,3.109577e+18,1.347137e+15,6.179102e+23,7.800662e+19,325347.0,2020-01-01,129.21,2020,1
1,5.072829e+18,1.576415e+15,1.118305e+24,1.712536e+20,361247.0,2020-01-02,130.75,2020,2
2,6.885988e+18,3.321197e+15,1.852351e+24,1.632737e+20,426976.0,2020-01-03,127.19,2020,3
3,4.459051e+18,2.000000e+15,1.132514e+24,1.544270e+20,396568.0,2020-01-04,134.30,2020,4
4,3.776137e+18,2.000000e+15,9.006654e+23,9.799816e+19,377354.0,2020-01-05,134.32,2020,5
...,...,...,...,...,...,...,...,...,...
361,6.043453e+18,1.385369e+16,3.404712e+24,2.467574e+20,991055.0,2020-12-27,636.35,2020,362
362,5.883312e+18,1.830259e+16,3.607847e+24,2.100049e+20,1054534.0,2020-12-28,683.76,2020,363
363,6.965450e+18,1.300000e+16,4.132441e+24,3.314315e+20,1020276.0,2020-12-29,730.26,2020,364
364,6.136375e+18,1.450000e+16,3.666141e+24,2.560363e+20,1045532.0,2020-12-30,732.04,2020,365


In [74]:
amoData_total_new = amoData_total_new[["meanValue", "medianValue", "sumValue", "stdValue","date","price","year","day","totaltx"]]
amoData_total_new

,meanValue,medianValue,sumValue,stdValue,date,price,year,day,totaltx
0,3.109577e+18,1.347137e+15,6.179102e+23,7.800662e+19,2020-01-01,129.21,2020,1,325347.0
1,5.072829e+18,1.576415e+15,1.118305e+24,1.712536e+20,2020-01-02,130.75,2020,2,361247.0
2,6.885988e+18,3.321197e+15,1.852351e+24,1.632737e+20,2020-01-03,127.19,2020,3,426976.0
3,4.459051e+18,2.000000e+15,1.132514e+24,1.544270e+20,2020-01-04,134.30,2020,4,396568.0
4,3.776137e+18,2.000000e+15,9.006654e+23,9.799816e+19,2020-01-05,134.32,2020,5,377354.0
...,...,...,...,...,...,...,...,...,...
361,6.043453e+18,1.385369e+16,3.404712e+24,2.467574e+20,2020-12-27,636.35,2020,362,991055.0
362,5.883312e+18,1.830259e+16,3.607847e+24,2.100049e+20,2020-12-28,683.76,2020,363,1054534.0
363,6.965450e+18,1.300000e+16,4.132441e+24,3.314315e+20,2020-12-29,730.26,2020,364,1020276.0
364,6.136375e+18,1.450000e+16,3.666141e+24,2.560363e+20,2020-12-30,732.04,2020,365,1045532.0


In [75]:
amoData_total_new.to_csv("trade/ethereum_2020_all_features.csv")

In [54]:
amoData_total_new

,0,1,2,3,4,Date,Open
0,3.109577e+18,1.347137e+15,6.179102e+23,7.800662e+19,325347.0,2020-01-01,129.21
1,5.072829e+18,1.576415e+15,1.118305e+24,1.712536e+20,361247.0,2020-01-02,130.75
2,6.885988e+18,3.321197e+15,1.852351e+24,1.632737e+20,426976.0,2020-01-03,127.19
3,4.459051e+18,2.000000e+15,1.132514e+24,1.544270e+20,396568.0,2020-01-04,134.30
4,3.776137e+18,2.000000e+15,9.006654e+23,9.799816e+19,377354.0,2020-01-05,134.32
...,...,...,...,...,...,...,...
361,NaN,NaN,NaN,NaN,NaN,2020-12-27,636.35
362,NaN,NaN,NaN,NaN,NaN,2020-12-28,683.76
363,NaN,NaN,NaN,NaN,NaN,2020-12-29,730.26
364,NaN,NaN,NaN,NaN,NaN,2020-12-30,732.04


In [49]:
amoData_total_new.set_index('date',inplace=True)
amoData_total_new

,meanValue,medianValue,sumValue,stdValue,totaltx,price
date,,,,,,
2020-01-01,6.219154e+18,2.694273e+15,1.235820e+24,1.560132e+20,325347.0,129.21
2020-01-02,1.014566e+19,3.152830e+15,2.236610e+24,3.425072e+20,361247.0,130.75
2020-01-03,1.377198e+19,6.642394e+15,3.704703e+24,3.265474e+20,426976.0,127.19
2020-01-04,8.918103e+18,4.000000e+15,2.265029e+24,3.088540e+20,396568.0,134.30
2020-01-05,7.552275e+18,4.000000e+15,1.801331e+24,1.959963e+20,377354.0,134.32
...,...,...,...,...,...,...
2020-12-27,1.208691e+19,2.770737e+16,6.809425e+24,4.935148e+20,991055.0,636.35
2020-12-28,1.176662e+19,3.660518e+16,7.215694e+24,4.200098e+20,1054534.0,683.76
2020-12-29,1.393090e+19,2.600000e+16,8.264882e+24,6.628631e+20,1020276.0,730.26


In [61]:
!zip -r ethOccMat.zip /root/study/ChainMT/data/eth/occMat/2020

  adding: root/study/ChainMT/data/eth/occMat/2020/ (stored 0%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-12-04.csv (deflated 57%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-06-03.csv (deflated 58%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-11-27.csv (deflated 57%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-06-18.csv (deflated 58%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-11-19.csv (deflated 56%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-04-17.csv (deflated 58%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-12-16.csv (deflated 57%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-10-29.csv (deflated 57%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-02-08.csv (deflated 58%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-01-09.csv (deflated 59%)
  adding: root/study/ChainMT/data/eth/occMat/2020/occ2020-07-03.csv (deflated 57%)
  adding: root/study/Cha